<h2 align="center">点击下列图标在线运行HanLP</h2>
<div align="center">
	<a href="https://colab.research.google.com/github/hankcs/HanLP/blob/doc-zh/plugins/hanlp_demo/hanlp_demo/zh/srl_mtl.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
	<a href="https://mybinder.org/v2/gh/hankcs/HanLP/doc-zh?filepath=plugins%2Fhanlp_demo%2Fhanlp_demo%2Fzh%2Fsrl_mtl.ipynb" target="_blank"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>
</div>

## 安装

无论是Windows、Linux还是macOS，HanLP的安装只需一句话搞定：

In [ ]:
!pip install hanlp -U

## 加载模型
HanLP的工作流程是先加载模型，模型的标示符存储在`hanlp.pretrained`这个包中，按照NLP任务归类。

In [1]:
import hanlp
hanlp.pretrained.mtl.ALL # MTL多任务，具体任务见模型名称，语种见名称最后一个字段或相应语料库

{'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_small_20201223_035557.zip',
 'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_base_20201223_201906.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20210111_124159.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_base_20210111_124519.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ERNIE_GRAM_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip',
 'UD_ONTONOTES_TOK_POS_LEM_FEA_NER_SRL_DEP_SDP_CON_MT5_SMALL': 'https://file.hankcs.com/hanlp/mtl/ud_ontonotes_tok_pos_lem_fea_ner_srl_dep_sdp_con_mt5_small_20210228_123458.zip',
 'UD_ONTONOTES_TOK_POS_LEM

调用`hanlp.load`进行加载，模型会自动下载到本地缓存。自然语言处理分为许多任务，分词只是最初级的一个。与其每个任务单独创建一个模型，不如利用HanLP的联合模型一次性完成多个任务：

In [2]:
HanLP = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH)

## 语义角色分析
任务越少，速度越快。如指定仅执行语义角色分析：

In [3]:
doc = HanLP('2021年HanLPv2.1为生产环境带来次世代最先进的多语种NLP技术。', tasks='srl')

返回值为一个[Document](https://hanlp.hankcs.com/docs/api/common/document.html):

In [4]:
print(doc)

{
  "tok/fine": [
    "2021年",
    "HanLPv2.1",
    "为",
    "生产",
    "环境",
    "带来",
    "次",
    "世代",
    "最",
    "先进",
    "的",
    "多",
    "语种",
    "NLP",
    "技术",
    "。"
  ],
  "srl": [
    [["2021年", "ARGM-TMP", 0, 1], ["HanLPv2.1", "ARG0", 1, 2], ["为生产环境", "ARG2", 2, 5], ["带来", "PRED", 5, 6], ["次世代最先进的多语种NLP技术", "ARG1", 6, 15]],
    [["最", "ARGM-ADV", 8, 9], ["先进", "PRED", 9, 10], ["技术", "ARG0", 14, 15]]
  ]
}


`doc['srl']`字段为语义角色标注结果，每个四元组的格式为`[论元或谓词, 语义角色标签, 起始下标, 终止下标]`。其中，谓词的语义角色标签为`PRED`，起止下标对应以`tok`开头的第一个单词数组。

可视化谓词论元结构：

In [5]:
doc.pretty_print()

Token    	SRL PA1     	Token    	SRL PA2     
─────────	────────────	─────────	────────────
2021年    	───►ARGM-TMP	2021年    	            
HanLPv2.1	───►ARG0    	HanLPv2.1	            
为        	◄─┐         	为        	            
生产       	  ├►ARG2    	生产       	            
环境       	◄─┘         	环境       	            
带来       	╟──►PRED    	带来       	            
次        	◄─┐         	次        	            
世代       	  │         	世代       	            
最        	  │         	最        	───►ARGM-ADV
先进       	  │         	先进       	╟──►PRED    
的        	  ├►ARG1    	的        	            
多        	  │         	多        	            
语种       	  │         	语种       	            
NLP      	  │         	NLP      	            
技术       	◄─┘         	技术       	───►ARG0    
。        	            	。        	            


遍历谓词论元结构：

In [6]:
for i, pas in enumerate(doc['srl']):
    print(f'第{i+1}个谓词论元结构：')
    for form, role, begin, end in pas:
        print(f'{form} = {role} at [{begin}, {end}]')

第1个谓词论元结构：
2021年 = ARGM-TMP at [0, 1]
HanLPv2.1 = ARG0 at [1, 2]
为生产环境 = ARG2 at [2, 5]
带来 = PRED at [5, 6]
次世代最先进的多语种NLP技术 = ARG1 at [6, 15]
第2个谓词论元结构：
最 = ARGM-ADV at [8, 9]
先进 = PRED at [9, 10]
技术 = ARG0 at [14, 15]


为已分词的句子执行语义角色分析：

In [7]:
HanLP([
    ["HanLP", "为", "生产", "环境", "带来", "次世代", "最", "先进", "的", "多语种", "NLP", "技术", "。"],
    ["我", "的", "希望", "是", "希望", "张晚霞", "的", "背影", "被", "晚霞", "映红", "。"]
  ], tasks='srl', skip_tasks='tok*').pretty_print()

Token	SRL PA1 	Token	SRL PA2     
─────	────────	─────	────────────
HanLP	───►ARG0	HanLP	            
为    	◄─┐     	为    	            
生产   	  ├►ARG2	生产   	            
环境   	◄─┘     	环境   	            
带来   	╟──►PRED	带来   	            
次世代  	◄─┐     	次世代  	            
最    	  │     	最    	───►ARGM-ADV
先进   	  │     	先进   	╟──►PRED    
的    	  ├►ARG1	的    	            
多语种  	  │     	多语种  	            
NLP  	  │     	NLP  	            
技术   	◄─┘     	技术   	───►ARG0    
。    	        	。    	            

Tok	SRL PA1 	Tok	SRL PA2 	Tok	SRL PA3 
───	────────	───	────────	───	────────
我  	◄─┐     	我  	        	我  	        
的  	  ├►ARG0	的  	        	的  	        
希望 	◄─┘     	希望 	        	希望 	        
是  	╟──►PRED	是  	        	是  	        
希望 	◄─┐     	希望 	╟──►PRED	希望 	        
张晚霞	  │     	张晚霞	◄─┐     	张晚霞	        
的  	  │     	的  	  │     	的  	        
背影 	  ├►ARG1	背影 	  │     	背影 	        
被  	  │     	被  	  ├►ARG1	被  	        
晚霞 	  │     	晚霞 	  │     	晚霞 	───►ARG0
映红 	◄─┘     	映红 	◄─┘ 

#### 注意
Native API的输入单位限定为句子，需使用[多语种分句模型](https://github.com/hankcs/HanLP/blob/master/plugins/hanlp_demo/hanlp_demo/sent_split.py)或[基于规则的分句函数](https://github.com/hankcs/HanLP/blob/master/hanlp/utils/rules.py#L19)先行分句。RESTful同时支持全文、句子、已分词的句子。除此之外，RESTful和native两种API的语义设计完全一致，用户可以无缝互换。